In [1]:
import pandas as pd
import numpy as np
from eng_module import utils

# Concrete

In [103]:
def extract_RAM_conc_column_data(filepath: str) -> dict[str, list[str]]:
    """
    Returns a dictionary of relevant column design data from the RAM Concrete
    Column "Column Design" csv.
    """
    raw_data = utils.read_csv_file(filepath)

    level = []
    grid_loc = []
    size = []
    rebar = []
    fpc = []
    lux = []
    luy = []
    kx = []
    ky = []
    pu = []
    mu_x_top = []
    mu_y_top = []
    mu_x_bot = []
    mu_y_bot = []

    for idx, row in enumerate(raw_data):
        # multiIndex keys
        if "Level." in row:
            level.append(row[1])
        if "Grid Location:." in row:
            grid_loc.append(row[-1])
            
        # corresponding values for above keys
        if "Size:." in row:
            item = row[1].split('  ')[0].strip()
            size.append(item)
        if "Longitudinal:." in row:
            rebar.append(row[1].strip().split(' ')[0])
        if "f'c (ksi):." in row:
            fpc.append(row[1].strip())
        if "Unbraced Length (ft)." in row:
            lux.append(row[1])
            luy.append(row[2])
        if "K." in row:
            kx.append(row[1].strip())
            ky.append(row[2].strip())
        if "Axial" in row:
            pu.append(row[-1])
        if "Moment" in row and "Top" in row:
            mu_x_top.append(row[-1].strip())
            mu_y_top.append(raw_data[idx+1][-1].strip())
        if "Moment" in row and "Bottom" in row:
            mu_x_bot.append(row[-1].strip())
            mu_y_bot.append(raw_data[idx+1][-1].strip())

    column_data = {"level": level,
                   "grid_loc": grid_loc,
                   "size": size,
                   "rebar": rebar,
                   "fpc": fpc,
                   "lux": lux,
                   "luy": luy,
                   "kx": kx,
                   "ky": ky,
                   "pu": pu,
                   "mu_x_top": mu_x_top,
                   "mu_y_top": mu_y_top,
                   "mu_x_bot": mu_x_bot,
                   "mu_y_bot": mu_y_bot
                  }
        
    print(len(level), len(grid_loc), len(size), len(rebar), len(fpc), len(lux),
          len(luy), len(kx), len(ky), len(pu), len(mu_x_top), len(mu_y_top),
          len(mu_x_bot), len(mu_y_bot))
            
    return column_data

In [114]:
def create_RAM_concrete_column_schedule(column_data: dict[str, list[str]], 
                    xlsx: bool = False, 
                    output_filename: str = "RAM_Concrete_Column_Schedule.xlsx") -> pd.DataFrame:
    """
    Returns a column schedule from the dictionary created from the 
    extract_RAM_conc_column_data() function.

    If xlsx = True, it will also produce an Excel file.
    """
    levels_and_loc_dict = {k: col_data[k] for k in ('level', 'grid_loc') if k in col_data}
    col_sched_dict = {k: col_data[k] for k in ('size', 'rebar', 'fpc', 'pu') if k in col_data}

    mi_idx = pd.MultiIndex.from_frame(pd.DataFrame(levels_and_loc_dict))
    col_sched_df = pd.DataFrame(col_sched_dict)
    col_sched_df.index = mi_idx

    col_sched_df = col_sched_df.transpose()

    final = []
    for story in pd.Series(col_data['level']).unique():
        story_df = col_sched_df.loc[:, story]
        story_df['story'] = story
        final.append(story_df)
    schedule_df = pd.concat(final)


    schedule_df = schedule_df.reindex(schedule_df.columns.sort_values(), axis=1)
    story = schedule_df.story
    schedule_df = schedule_df.drop(columns=['story'])
    schedule_df.insert(loc=0, column='story', value=story)

    if xlsx:
        schedule_df.to_excel(output_filename)
    
    return schedule_df

In [115]:
col_data = extract_RAM_conc_column_data(r'./test_data/17605_20240328_dk_col_design.csv')

1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211


In [116]:
schedule = create_RAM_concrete_column_schedule(col_data, False)

/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_67055/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_67055/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_67055/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [117]:
schedule.head()

grid_loc,story,(-0.50ft-18.84ft),(-0.58ft--0.58ft),(1.02ft-90.91ft),(1.21ft-100.17ft),(1.31ft-104.84ft),(110.46ft-105.17ft),(110.52ft--1.72ft),(110.53ft-100.20ft),(110.53ft-18.85ft),...,(74.61ft--1.72ft),(90.61ft-73.87ft),(90.64ft-76.40ft),(90.83ft-90.91ft),(90.94ft-100.20ft),(91.01ft-105.17ft),(91.44ft--1.72ft),(91.44ft-18.85ft),(91.44ft-32.83ft),(91.44ft-46.16ft)
size,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rebar,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fpc,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pu,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size,BULKHEAD,16 x 24,16 x 24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16 x 24,16 x 24,NaN,NaN,NaN,NaN,16 x 24,16 x 24


# Steel